In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/data_fakenews_classifier/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df = df.dropna()

In [4]:
X = df.drop('label', axis = 1)
y = df['label']

In [5]:
X.shape

(18285, 4)

In [6]:
y.shape

(18285,)

In [7]:
import tensorflow as tf

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Init Plugin
Init Graph Optimizer
Init Kernel


In [8]:
tf.__version__

'2.5.0'

In [35]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [10]:
voc_size = 5000

#### One Hot Representation

In [11]:
messages = X.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
from nltk.corpus import stopwords
import re

In [14]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

corpus = []

for row in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][row])
    review = review.lower()
    review = review.split()

    review = [stemmer.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [15]:
onehot_repre = [one_hot(words,voc_size) for words in corpus]
onehot_repre

[[2557, 2045, 3994, 1382, 1464, 4353, 4004, 1761, 1686, 476],
 [2576, 4128, 4776, 1720, 2436, 1324, 4417],
 [1571, 389, 4218, 2314],
 [2624, 3286, 1317, 1851, 4638, 2152],
 [2164, 2436, 3146, 4488, 2435, 3578, 2436, 3228, 942, 3462],
 [2188,
  2603,
  236,
  866,
  4248,
  3619,
  3281,
  1690,
  2750,
  1107,
  4904,
  3025,
  185,
  494,
  4417],
 [1035, 1856, 3908, 3826, 4110, 2362, 4564, 3704, 1433, 1321, 2686],
 [1200, 4032, 635, 4679, 2275, 3546, 3619, 1947, 1433, 1321, 2686],
 [2821, 2313, 747, 947, 2378, 367, 485, 1789, 3619, 1166],
 [145, 2125, 2042, 3956, 3129, 3275, 202, 259],
 [1708, 1792, 933, 2025, 4933, 731, 862, 86, 3365, 3629, 4721],
 [1851, 509, 1464, 367, 3619, 2275],
 [1344, 1014, 331, 1628, 3219, 1095, 4433, 2031, 4354],
 [2412, 2307, 608, 2073, 1338, 356, 3609, 1433, 1321, 2686],
 [2349, 413, 3122, 1342, 3830, 1433, 1321, 2686],
 [64, 636, 2266, 4431, 989, 3565, 4451, 4052, 593, 4063],
 [2377, 838, 4128],
 [4140, 3897, 3630, 4459, 3619, 920, 3115, 4417],
 [37, 303

#### Embedding Representation

In [16]:
sent_len = 20
embedded_docs = pad_sequences(onehot_repre, padding="pre",maxlen=sent_len)
embedded_docs

array([[   0,    0,    0, ..., 1761, 1686,  476],
       [   0,    0,    0, ..., 2436, 1324, 4417],
       [   0,    0,    0, ...,  389, 4218, 2314],
       ...,
       [   0,    0,    0, ..., 1433, 1321, 2686],
       [   0,    0,    0, ..., 2272, 3386, 2086],
       [   0,    0,    0, ...,  706,    8,  655]], dtype=int32)

In [17]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2557,
       2045, 3994, 1382, 1464, 4353, 4004, 1761, 1686,  476], dtype=int32)

In [27]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
model.summary()

In [19]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [23]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=0)

#### Model Training

In [26]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

#### Adding Dropout

In [40]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss ='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Performance Metrics and Accuracy

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
accuracy_score(y_test,y_pred)